# Video understanding - 소셜 미디어 비디오를 위한 콘텐츠 조정
Video Understanding을 사용한 소셜 미디어 비디오 콘텐츠 조정은 AI를 활용하여 폭력, 노출 또는 유해한 행동과 같은 안전하지 않거나 부적절한 콘텐츠를 대량의 사용자 생성 비디오에서 자동으로 감지합니다. 오디오 및 시각적 신호를 모두 분석함으로써 플랫폼이 커뮤니티 표준을 유지하고, 수동 검토 노력을 줄이며, 대규모로 청중을 보호할 수 있도록 합니다.

이 샘플 노트북에서는 Video Understanding tool로 추출한 메타데이터를 사용합니다. 이 메타데이터에는 timestamp가 있는 shot 수준 조정 결과 및 오디오 transcript가 포함되어 있습니다. 이 메타데이터를 검색하고 더 깊은 조정 분석을 위해 LLM을 적용합니다.
![content_moderation](../statics/video-content-moderation.png)

In [ ]:
from utils import s3_tool, bedrock_tool
import json
import boto3
from IPython.display import JSON, Markdown, display

Video Understanding Tool의 결과는 Amazon S3와 Amazon DynamoDB 모두에 저장되며, 동일한 AWS 계정 내에서 직접 액세스할 수 있습니다.

이 lab에서는 `utils.s3_tool`의 함수를 사용하여 S3에서 추출된 비디오 요약 및 오디오 transcription을 검색합니다. 이 tool은 솔루션에서 프로비저닝한 S3 bucket에 추출 파일을 저장하며, 다음 명명 규칙을 따릅니다: `bedrock-mm-{account_id}-{region}`.

모든 추출된 정보는 데이터 bucket 내의 `tasks` prefix 아래에 구성되며, 각 task의 데이터는 task ID에 해당하는 별도의 하위 prefix에 저장됩니다. S3 bucket을 검토하여 모든 추출된 정보를 탐색할 수 있습니다.

In [ ]:
# 데이터 bucket 출력
session = boto3.session.Session()
sts = session.client("sts")
account_id = sts.get_caller_identity()["Account"]
region = session.region_name

print(f"bedrock-mm-{account_id}-{region}")

Video Understanding Tool UI에서 task ID를 찾을 수 있습니다.

Shot-based 섹션으로 이동하여 이전에 UI를 사용하여 처리한 콘텐츠 조정 샘플 비디오를 선택합니다. Get Task ID 링크를 클릭한 다음 아래 값을 해당 값으로 바꿉니다.

![Find task Id](../statics/find-task-id-moderation.png)


In [ ]:
task_id = 'YOUR_TASK_ID_FROM_VIDEO_UNDERSTANDING_TOOL'

Video Understanding Tool의 관리형 데이터베이스에서 transcript를 검색합니다.

In [ ]:
transcripts = s3_tool.get_transcripts(task_id)
print(json.dumps(transcripts,indent=2, ensure_ascii=False))

Video Understanding Tool의 관리형 데이터베이스에서 shot 요약을 검색합니다.

In [ ]:
shots = s3_tool.get_shot_outputs(task_id=task_id, output_names=["Summarize shot","Content Moderation","Detect text"])
print(json.dumps(shots,indent=2, ensure_ascii=False))

## shot 조정 감지 결과 분석
콘텐츠 조정 사용 사례의 경우, 각 비디오 shot 또는 frame을 분석하고 더 쉬운 의사 결정을 위해 조정 레이블을 할당하도록 프롬프트를 사용자 정의할 수 있습니다. 이 샘플에서는 LLM에게 시각적 및 오디오 입력을 모두 고려하여 비디오가 안전하지 않은지에 대한 정보에 입각한 결정을 내리도록 요청합니다.

In [ ]:
prompt = f'''
당신은 콘텐츠 조정 전문가입니다. 당신의 작업은 shot 수준 조정 레이블과 오디오 transcript를 모두 분석하여 비디오가 청중에게 안전한지 확인하는 것입니다.
다음 카테고리에 대해 비디오를 평가하십시오:
- 알코올: 알코올 소비와 관련된 장면 또는 오디오 참조.
- 담배: 담배 또는 흡연과 관련된 장면 또는 오디오 참조.
- 선정적: 노출, 상반신 노출 남성, 노출이 심한 의상 또는 기타 성적으로 선정적인 자료와 같은 콘텐츠.
- 비속어: 혐오 발언, 괴롭힘 또는 노골적인 언어를 포함한 오디오의 공격적이거나 유해한 언어.

지침:
시각적(shot 수준 조정 레이블)과 오디오(transcript)를 모두 고려하십시오.
비디오에 위의 카테고리 중 하나가 포함되어 있는지 명확하게 명시하십시오.

Shot 요약 및 조정 레이블:
{shots}
오디오 Transcript:
{transcripts}
'''

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id="global.amazon.nova-2-lite-v1:0", 
    prompt=prompt, 
    inference_config={
      "maxTokens": 10000,
      "topP": 0.1,
      "temperature": 0.3
    }
)
result = bedrock_tool.parse_converse_response(response)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(result.replace('\\n', '\n').replace('\\"', '"')))